In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 5.1MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

In [0]:
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv("data/men_shoes.csv", low_memory=False)

In [9]:
df.shape

(18280, 48)

In [10]:
df.values

array([['AVpfHrJ6ilAPnD_xVXOI', nan, 'Josmo', ..., nan, nan, nan],
       ['AVpfHrJ6ilAPnD_xVXOI', nan, 'Josmo', ..., nan, nan, nan],
       ['AVpfHsWP1cnluZ0-eVZ7', nan, 'SERVUS BY HONEYWELL', ..., nan,
        nan, nan],
       ...,
       ['AVpf0fJXLJeJML43EVe9', nan, 'Seek Optics', ..., nan, nan, nan],
       ['AVpf0fJXLJeJML43EVe9', nan, 'Seek Optics', ..., nan, nan, nan],
       ['AVpf0fJXLJeJML43EVe9', nan, 'Seek Optics', ..., nan, nan, nan]],
      dtype=object)

In [11]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [94]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
feats1 = ['brand_cat']
x = df[ feats1 ].values
y = df[ 'prices_amountmin' ].values

model = DecisionTreeRegressor(max_depth=5)

scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-58.133398968282776, 4.206122611474276)

In [0]:
def run_model(feast1, model=DecisionTreeRegressor(max_depth=5)):
  x = df[ feats1 ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [39]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [40]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [41]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [43]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [44]:
test = {'key': 'value'}
test['key']

'value'

In [45]:
str(test)

"{'key': 'value'}"

In [49]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)[0]['key']

['Men']

In [50]:
literal_eval(str_dict)[0]['value']

['Men']

In [51]:
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict


df['features_parsed'] = df['features'].map(parse_features)

In [63]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [59]:
[
 {'key': 'Gender', 'value': ['Men']}, 
 {'key': 'Shoe Size', 'value': ['M']}, 
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, 
 {'key': 'Color', 'value': ['Multicolor']}, 
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, 
 {'key': 'Brand', 'value': ['Josmo']}]

{
    'Gender': 'Men',
    'Shoe Size': 'M',
    
}

{'Gender': 'Men', 'Shoe Size': 'M'}

In [64]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

0        None
1        None
2        None
3        None
4        None
         ... 
18275    None
18276    None
18277    None
18278    None
18279    None
Name: features_parsed, Length: 18280, dtype: object

In [65]:
len(keys)

476

In [67]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [70]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats1: feats1[key] if key in feats1 else np.nan)

In [71]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_fabric type', 'feat_sock size', 'feat_movement',
       'feat_water resistant', 'feat_cm', 'feat_stylenumber', 'feat_gender',
       'feat_hands', 'feat_eye size:', 'feat_country of manufacture'],
      dtype='object', length=526)

In [82]:
df[ False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] * 100

0.0437636761487965

In [78]:
df.shape[0]

18280

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [93]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [102]:
df [ df.brand == df.feat_brand ].shape

(52, 531)

In [103]:
df [ df.brand != df.feat_brand ].shape

(18228, 531)

In [107]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df [ df.brand != df.feat_brand ][ ['brand', 'feat_brand'] ].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [113]:
df [ df.brand == df.feat_brand ].shape

(8846, 531)

In [114]:
run_model(['brand_cat'] , model )

(-57.325021113018806, 4.161796247293468)

In [0]:
feats3 = ['']

In [117]:
feats4 = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model( feats4 , model )

(-57.299871305693344, 4.169463596292802)

In [118]:
x = df[ feats4 ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(x, y)

perm = PermutationImportance(m, random_state=1).fit(x, y);
eli5.show_weights(perm, feature_names = feats4)

Weight,Feature
0.2565 ± 0.0110,brand_cat
0.1057 ± 0.0095,feat_material_cat
0.0431 ± 0.0043,feat_gender_cat
0.0204 ± 0.0010,feat_brand_cat
0.0072 ± 0.0004,feat_color_cat
0.0008 ± 0.0001,feat_manufacturer part number_cat


In [0]:
feats5 = ['brand_cat', 'feat_brand_cat', 'feat_shape_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat',  'feat_metal type_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats5, model)

In [157]:
x = df[ feats5 ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(x, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(x, y);
eli5.show_weights(perm, feature_names = feats5)

(-57.35300718195678, 4.164933380982146)


Weight,Feature
0.2585 ± 0.0111,brand_cat
0.1025 ± 0.0149,feat_material_cat
0.0245 ± 0.0029,feat_gender_cat
0.0171 ± 0.0011,feat_brand_cat
0.0129 ± 0.0011,feat_shape_cat
0.0091 ± 0.0010,feat_metal type_cat
0.0034 ± 0.0006,feat_style_cat


In [147]:
df['brand'].value_counts(normalize=True)

nike                         0.097210
puma                         0.033315
ralph lauren                 0.028775
vans                         0.021116
new balance                  0.020295
                               ...   
earrings-midwestjewellery    0.000055
bugatchi                     0.000055
american rag                 0.000055
hue                          0.000055
dingo                        0.000055
Name: brand, Length: 1732, dtype: float64

In [145]:
df[ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [154]:
df[ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'type': 'cleats', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': 'basketball', 'fabric content': '100% synthetic', 'color': 'sport royal/game royal', 'model': '314996-401', 'casual & dress shoe style': 'basketball shoes', 'brand': 'nike', 'manufacturer_part_number': '314996-401-9-5'},
       {'style': 'running, cross training', 'pattern': 'multi-colored', 'material': 'synthetic', 'country/region of manufacture': 'vietnam', 'color': 'blue', 'condition': 'new with box'},
       {'type': 'cleats', 'condition': 'new with box'},
       {'sport': 'leisure, golf', 'style': 'pants', 'country/region of manufacture': 'china', 'condition': 'new with tags'}],
      dtype=object)

In [0]:
!git add matrix_one/day5.ipynb

In [161]:
!git commit -m "1st ML for Men's Shoe Prices - day 5 "


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@691035e6f10d.(none)')


In [0]:
!git config --global user.email "lukasz.jaspinski@gmail.com"
!git config --global user.name "Lukasz"

In [163]:
!git push -u origin master

Branch 'master' set up to track remote branch 'master' from 'origin'.
Everything up-to-date
